# Модуль 5.4 — Мини‑деплой (FastAPI)

**Цель:** показать, как превратить цепочку в HTTP‑сервис.

**Что сделаем:**
- установим FastAPI
- создадим эндпоинт `/ask`
- проверим запрос через `requests`

In [ ]:
%pip -q install -U \
  fastapi \
  uvicorn \
  langchain \
  langchain-openai \
  python-dotenv \
  pydantic==2.12.3 \
  requests==2.32.4

## Настройка провайдера (AITunnel)

Укажем ключ и базовый URL для вызовов модели.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Создаём простой API

Эндпоинт `/ask` принимает вопрос и возвращает ответ.

In [ ]:
import textwrap
from pathlib import Path

app_code = textwrap.dedent('''
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

app = FastAPI()

llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
)

prompt = PromptTemplate.from_template(
    "Ответь коротко и по делу: {question}"
)

class AskRequest(BaseModel):
    question: str

@app.post("/ask")
def ask(req: AskRequest):
    chain = prompt | llm
    result = chain.invoke({"question": req.question})
    return {"answer": result.content}
''')

Path("app.py").write_text(app_code)
print("app.py создан")

## Запуск сервера

В Colab используем `&`, чтобы сервер не блокировал ноутбук.

In [ ]:
# Запуск в фоне
!uvicorn app:app --host 0.0.0.0 --port 8000 &

## Тестируем запрос

In [ ]:
import time
import requests

time.sleep(1)

resp = requests.post(
    "http://localhost:8000/ask",
    json={"question": "Что такое RAG?"},
)
print(resp.json())